# Punto 4 Recuperación ranqueada y vectorización de documentos (RRDV) GENSIM
## Integrantes
* Juan Esteban Arboleda
* Luccas Rojas

### 1. Preprocesamiento
Lo primero que se llevara a cabo para poder hacer la recuperación ranqueada es la tokenizacion de los documentos y generacion del vocabulario. Ademas es importante tener en cuenta que el vocabulario debe estar ordenado, no debe contener stop-words, debe estar stemizado y normalizado

* A continucion se cargan los documentos y los queries en una estructura de datos, se debe cambiar DOCUMENTS_PATH y QUERIES_PATH por la ruta donde se encuentran los documentos y los queries respectivamente
* El archivo de salida es guardado en RRDV_RESULTS_FILE_PATH con el formato exigido

In [12]:
import os
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math
import time
from gensim import corpora, models

# Rutas a definir segun la ubicacion de los archivos
DOCUMENTS_PATH = '../data/docs-raw-texts'
QUERIES_PATH = '../data/queries-raw-texts'
GENSIM_RESULTS_FILE_PATH = "../data/GENSIM-consultas_resultados"

def load_documents(folder_path: str) -> pd.DataFrame:
    """
    Returns a Pandas DataFrame where each row represents a document in folder_path.
    The DataFrame will have as many rows as there are documents in folder_path

        Parameters
        ----------
            folder_path: str
                The path to the folder that contains the documents to load
    
        Returns
        --------
            documents: pd.DataFrame
                Pandas DataFrame with two columns: "filename" and "body"
    """
    documents = []
    index = []
    id = 1
    columns = ['filename', 'body']
    for filename in os.listdir(folder_path):
        text = pd.read_xml(os.path.join(folder_path, filename))['raw'].tolist()[1]
        filtered_text = text.replace('\n', ' ').replace('\xa0', ' ')
        document = [filename, filtered_text]
        documents.append(document)
        index.append(id)
        id += 1

    return pd.DataFrame(documents, index, columns)

documents = load_documents(DOCUMENTS_PATH)
queries = load_documents(QUERIES_PATH)

documents

,filename,body
1,wes2015.d001.naf,William Beaumont and the Human Digestion. Wil...
2,wes2015.d002.naf,Selma Lagerlöf and the wonderful Adventures of...
3,wes2015.d003.naf,Ferdinand de Lesseps and the Suez Canal. Ferd...
4,wes2015.d004.naf,Walt Disney’s ‘Steamboat Willie’ and the Rise ...
5,wes2015.d005.naf,Eugene Wigner and the Structure of the Atomic ...
...,...,...
327,wes2015.d327.naf,James Parkinson and Parkinson’s Disease. Wood...
328,wes2015.d328.naf,Juan de la Cierva and the Autogiro. Demonstra...
329,wes2015.d329.naf,Squire Whipple – The Father of the Iron Bridge...
330,wes2015.d330.naf,William Playfair and the Beginnings of Infogra...


Primero que todo tokenizamos el texto, para esto utilizamos el word tokenize de la libreria NLTK

In [13]:
nltk.download('punkt')
nltk.download('stopwords')

documents['tokens'] = documents['body'].apply(word_tokenize)
queries['tokens'] = queries['body'].apply(word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Removemos todos los signos de puntuacion, contracciones del ingles y dejamos el texto todo en minusculas (normalizar) 

In [14]:
def remove_punctuation(token_list):
    return [token.lower() for token in token_list if (token not in string.punctuation and (len(token)>1 or token.isnumeric()))]

documents['tokens']=documents['tokens'].apply(lambda x: remove_punctuation(x))
queries['tokens']=queries['tokens'].apply(lambda x: remove_punctuation(x))

Luego de tokenizar, dejar todo en minusculas, quitaremos las stop words para que reduzcan el vocabulario y no afecten el resultado final. Para esto usaremos la libreria nltk y su metodo stopwords.words('english').

In [15]:
stop_words = set(stopwords.words('english'))

#TODO no se si normalizar cuente como poner todo en minusculas
def remove_stop_words(token_list):
    return [token for token in token_list if token not in stop_words]

documents['tokens']=documents['tokens'].apply(lambda x: remove_stop_words(x))
queries['tokens']=queries['tokens'].apply(lambda x: remove_stop_words(x))

Luego de eliminar las stop words se hace stemming a las palabras restantes.

In [16]:
stemmer = PorterStemmer()
def stemming(token_list):
    return [stemmer.stem(token) for token in token_list]

documents['tokens']=documents['tokens'].apply(lambda x: stemming(x))
queries['tokens']=queries['tokens'].apply(lambda x: stemming(x))

En este punto el texto de cada documento y query esta en un formato mas facil de procesar, por lo que se procede a realizar la representacion vectorial de los documentos y queries.

## 2. Representación de los datos

Primero se crea un diccionario y el corpus

In [17]:
dictionary = corpora.Dictionary(documents["tokens"])
corpus = [dictionary.doc2bow(text) for text in documents["tokens"]]

Se crea el modelo TF-IDF a partor del corpus

In [19]:
tfidf = models.TfidfModel(corpus)

Se calcula el tf-idf para el corpus

In [20]:
corpus_tfidf = tfidf[corpus]

Se crea la matriz de similitudes, con respecto a la cual se puede consultar la similitud de un documento con respecto a todos los documentos del corpus

In [21]:
from gensim import similarities
index = similarities.MatrixSimilarity(corpus_tfidf)

# 3. Evaluación

Se realiza la similitud coseno entre las consultas y el corpus y se escribe el archivo de resultados

In [27]:
# Clear output file contents
open(GENSIM_RESULTS_FILE_PATH, "w").close()
file = open(GENSIM_RESULTS_FILE_PATH, "a")

# Loop through queries
for i, query in queries.iterrows():
    # Open output file
    query_str = query['filename'].replace('.naf', '').replace('wes2015.', '')
    file.write(query_str + " ")

    # Perform cosine similarity between corpus ans query
    query_bow = dictionary.doc2bow(query["tokens"])
    query_tfidf = tfidf[query_bow]
    sims = index[query_tfidf]
    sims = sorted(enumerate(sims, start=1), key=lambda item: item[1], reverse=True)


    # Write output file
    i = 0
    for docId, sim in sims:
        if(sim > 0):
            if docId < 10:
                doc_str = "d00" + str(docId)
            elif docId < 100:
                doc_str = "d0" + str(docId)
            else:
                doc_str = "d" + str(docId)
            file.write(doc_str + ":" + str(sim))
            if i != len(sims):
                file.write(",")
        i += 1

    if query_str != "q46":
        file.write("\n")

file.close()